# *Clustering* with K-means

This notebook is a modified version of [Professor Cristina Gómez's more detailed notebook](https://github.com/cristinagom/machinelearning/blob/main/ejercicios/UD3-UnsupervisedLearning/ML_3.1_UnsupervisedLearning_Clustering/ML_3_1_UnsupervisedLearning_Clustering.ipynb).

## Introduction

***Clustering*** is an **unsupervised learning** technique that groups records into groups of similar characteristics, known as ***clusters***.

**K-means** is a clustering algorithm that groups data by attempting to separate samples into $K$ predefined groups, where each sample belongs to the group whose mean (the cluster **centroid**) is closest.

The groups are established by minimizing the **inertia or WCSS (Within-Cluster Sum of Squares)**, which is the sum of squared distances from each sample to the centroid of its group.

The K-means algorithm follows these steps:
1. **Initialization**: Choose $K$ initial centroids randomly or using some specific method.
2. **Assignment**: Assign each sample to the nearest centroid.
3. **Update**: Recalculate the centroids as the mean of the samples in each group.
4. **Iteration**: Repeat steps 2 and 3 until the centroids no longer change (the algorithm converges) or a maximum number of iterations is reached.

[![K-means](img/K-means_convergence.gif)](https://upload.wikimedia.org/wikipedia/commons/e/ea/K-means_convergence.gif)

## Creating Test Data

We will use the `make_blobs` function from `sklearn.datasets` to generate test data. This function generates a dataset of points distributed as point clouds. We pass the following parameters:
- `n_samples`: total number of samples
- `n_features`: number of features for each sample. We will work with only 2 features to be able to visualize the data in 2D.
- `centers`: number of clusters we want to generate
- `random_state`: seed for random number generation
- `cluster_std`: standard deviation of the clusters

It's important to remember that we are doing this representation process to help understand the algorithm, but we will normally work with datasets of many more dimensions where we cannot visualize the data. In fact, in this case it would normally be easy to divide into clusters by eye. And indeed, we are already generating test data with this `make_blobs` function by indicating the number of clusters we want it to generate.

In [ ]:
from sklearn import datasets

X, y = datasets.make_blobs(n_samples=1000, # Number of samples
                           n_features=2, # Number of features
                           centers=5, # Number of clusters
                           cluster_std=[0.5, 0.6, 0.8, 1, 1.1], # Standard deviation of each cluster
                           random_state=42,) # Seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X[:, 0], X[:, 1], s=2)
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

## Training K-means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=5, # Number of clusters (k value)
    n_init='auto') # Number of times the algorithm runs with different initial centroids
y_pred = kmeans.fit_predict(X) # Equivalent to kmeans.fit(X) and then y_pred = kmeans.predict(X)

The KMeans constructor receives the following parameters:
* `n_clusters`: The number of clusters to form and the number of centroids to generate. This is what we vary in a loop to find the optimal number of clusters.
* `init`: Method for initializing centroids. The simplest is 'random', but by default it uses 'k-means++', which is a more sophisticated method that tries to place initial centroids far apart from each other.
* `max_iter`: Maximum number of iterations of the algorithm for a single run.
* `n_init`: Number of times the algorithm will be run with different centroids. The final result will be the best output of n_init consecutive runs in terms of inertia.

Now, with the WCSS array, we can use the elbow method to determine the optimal number of clusters. The elbow method is based on the variation of inertia (WCSS) as a function of the number of clusters. If we increase the number of clusters, inertia will decrease. The goal is to choose a number of clusters where inertia decreases significantly.

Like other `scikit-learn` estimators, KMeans has a `fit_predict` method that can be used to train the model and predict the clusters to which the data belongs. This method returns an array of cluster labels, which assigns each data point to a cluster, and also stores it in the `labels_` property of the KMeans object.

With the `is` operator we can check if two variables point to the same memory address, that is, if they are the same object. In this case, we are checking if the `labels_` attribute of the KMeans object is the same as the label array returned by the `fit_predict` method.

In [ ]:
y_pred is kmeans.labels_ 

The cluster centroids can be obtained with the `cluster_centers_` attribute.

In [ ]:
print(kmeans.cluster_centers_)
plt.scatter(X[:, 0], X[:, 1], s=2)
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c='r', s=30)

for i, center in enumerate(kmeans.cluster_centers_):
    plt.annotate(i, center, fontsize=18)

plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

We can easily predict the cluster to which each new instance belongs with the `predict` method:

In [ ]:
import numpy as np
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, -2.5], [-3, 5]])
y_new = kmeans.predict(X_new)
y_new

In [ ]:
print(kmeans.cluster_centers_)
plt.scatter(X[:, 0], X[:, 1], s=2)
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c='r', s=20)
plt.scatter(X_new[:, 0], X_new[:, 1], c='g', s=50)

for i, center in enumerate(kmeans.cluster_centers_):
    plt.annotate(i, center, fontsize=18)
    
for i, point in enumerate(X_new): # Annotation for new points
    plt.annotate(f'x{i}={y_new[i]}', point, fontsize=12)

plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

By plotting the decision boundaries of the algorithm, we obtain a [Voronoi tessellation](https://en.wikipedia.org/wiki/Voronoi_diagram):

In [ ]:
def plot_decision_regions(kmeans, X):
  x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1 # Min and max values for x1
  y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1 # Min and max values for x2
  xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), 
                      np.arange(y_min, y_max, 0.1)) 
  
  Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)

  plt.figure(figsize=(8, 8))
  plt.contourf(xx, yy, Z, alpha=0.4)
  plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=20, edgecolor='k')
  plt.show()
  
plot_decision_regions(kmeans,X)

The vast majority of instances were clearly assigned to their original cluster.

The only thing K-means cares about is the distance between instances and centroids. Instead of assigning each instance to a cluster (hard clustering), it's better to give a cluster score per instance (soft clustering). The score can be the distance between the instance and the centroids.

The `transform` method measures the distance between each instance and the centroids.

In [ ]:
kmeans.transform(X_new)

## Choosing the Number of Clusters (*Elbow Method*)

We might think we can choose the model with the lowest inertia. This poses a problem because increasing `k` will always give us lower inertia (or distortion).

The elbow method is a heuristic used to find the optimal number of clusters in a clustering algorithm. The idea is to run it in a loop calculating inertia and choose the value of k where inertia stops decreasing rapidly.

Let's visualize inertia as a function of `k`:

<img src="img/k_to_inertia.png" width="700" />

As we can see, distortion (or inertia) drops significantly when going from $3$ to $4$, but then decreases much more slowly as we continue increasing $k$. This curve has roughly the shape of an arm with an elbow at $k=4$.

However, it should be noted that this method is somewhat subjective and depends on the shape of the data.

In [ ]:
inertia = []
K = range(1,10)
for k in K:
    model = KMeans(k).fit(X)
    inertia.append(model.inertia_)

In [ ]:
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Finding the Optimal K using the Elbow Method')
plt.show()

## Limitations of K-means

- `K-means` is not perfect, so it is necessary to run the algorithm several times to avoid suboptimal solutions.
- Another limiting factor of the algorithm is that we need to specify the number of clusters.
- `K-means` also doesn't perform very well when groups have different sizes, different densities, or non-spherical shapes.
- Depending on the data, different clustering algorithms may work better (such as `DBSCAN` or `Gaussian Mixtures`).
- Scaling the inputs with a StandardScaler is also essential with `K-means`.

## Additional Resources

- https://scikit-learn.org/stable/modules/clustering.html#k-means
- https://www.datacamp.com/tutorial/k-means-clustering-python
- [StatQuest: K-means clustering](https://www.youtube.com/watch?v=4b5d3muPQmA)